In [11]:
import sys
sys.path.append("src")

In [3]:
import os

In [4]:
pwd

'c:\\Users\\Meow\\Desktop\\PORTFOLIO\\MammoScan\\research'

In [5]:
os.chdir("../")

In [6]:
pwd

'c:\\Users\\Meow\\Desktop\\PORTFOLIO\\MammoScan'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
import sys
print(sys.path)


['c:\\Users\\Meow\\.conda\\envs\\mammoenv\\python39.zip', 'c:\\Users\\Meow\\.conda\\envs\\mammoenv\\DLLs', 'c:\\Users\\Meow\\.conda\\envs\\mammoenv\\lib', 'c:\\Users\\Meow\\.conda\\envs\\mammoenv', '', 'C:\\Users\\Meow\\AppData\\Roaming\\Python\\Python39\\site-packages', 'c:\\Users\\Meow\\.conda\\envs\\mammoenv\\lib\\site-packages', 'c:\\Users\\Meow\\.conda\\envs\\mammoenv\\lib\\site-packages\\win32', 'c:\\Users\\Meow\\.conda\\envs\\mammoenv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Meow\\.conda\\envs\\mammoenv\\lib\\site-packages\\Pythonwin']


In [13]:
!pip show PyYAML


In [14]:
pip install PyYAML

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.1.11 requires matplotlib>=3.3.0, which is not installed.
ultralytics 8.1.11 requires numpy>=1.22.2, which is not installed.
ultralytics 8.1.11 requires opencv-python>=4.6.0, which is not installed.
ultralytics 8.1.11 requires pandas>=1.1.4, which is not installed.
ultralytics 8.1.11 requires pillow>=7.1.2, which is not installed.
ultralytics 8.1.11 requires py-cpuinfo, which is not installed.
ultralytics 8.1.11 requires scipy>=1.4.1, which is not installed.
ultralytics 8.1.11 requires seaborn>=0.11.0, which is not installed.


In [16]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [18]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [19]:

import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-15 03:42:56,647: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-15 03:42:56,653: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-15 03:42:56,657: INFO: common: created directory at: artifacts]
[2025-07-15 03:42:56,657: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-15 03:42:56,665: INFO: 1983665874: Downloading data from https://drive.google.com/file/d/1w_OVqKtU0QGTm3nGo5A0-eFo6f3Cc9rY/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1w_OVqKtU0QGTm3nGo5A0-eFo6f3Cc9rY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1w_OVqKtU0QGTm3nGo5A0-eFo6f3Cc9rY&confirm=t&uuid=bcaceff0-0816-4d65-8b4a-dad34d5cb2e4
To: c:\Users\Meow\Desktop\PORTFOLIO\MammoScan\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:15<00:00, 3.24MB/s]

[2025-07-15 03:43:15,286: INFO: 1983665874: Downloaded data from https://drive.google.com/file/d/1w_OVqKtU0QGTm3nGo5A0-eFo6f3Cc9rY/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
